# CitrineChallenge Review
---


This notebook is presented to offer a more in-depth review of the CitrineChallenge library. Specifically, to address the question, "why do you believe this works well".

#### To Note:
this report is interactive. Run the code cells top to bottom to make full use of the notebook.

---

To begin, we should make explicit what a well-working submission should accomplish. Here we have several objectives, as stated in the challenge prompt and in dialogue with the team leader:

<br>

- All the produced vectors satisfy all the constraints.

- The vectors cover as much of the valid space as possible.

- Execution should run faster than 5 minutes.

- Code must be maintainable.

- Code must be extensible.

- Installation should be user-friendly and through Git.

- Solution should be generated start-to-finish in under 10 hours.

<br>

The solution presented in the CitrineChallenge addresses all of these objectives to varying degrees of success, which we'll discuss below.

---

To start, making the implementation seamless for users is simply a matter of understanding how to build a setup function. It should always be an aim to build implementations which are as semaless as possible, which is what I feel we did here with the `pip + git` installation process.

In [0]:
pip install git+https://github.com/1mikegrn/CitrineChallenge

This includes mandating install requirements and building in a console script entry point.

In [0]:
import CitrineChallenge
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
import requests
import time

When it comes to a maintainable and extensible library system, one of the requirements I've found to be key to meeting these objectives is modularity and documentation. If a library is to be maintainable, it needs to have the ability to flex with time as new tools become available for integration, or existing tools require update. Modularity is key here. Say a team leader isn't satisfied with how new points are generated. Modularity allows a team to go into the library, find the module which is in charge of generating the data points, and tinker with it as necessary.

This segues well into my commitment to thorough documentation. As Guido va Rossum has noted, *'Code is read more often than it is written'*. And to a further point, code systems become more difficult to maintain and update as it becomes more difficult for a user to figure out what a code segments objective is. This is why documentation is also key. Keying off our previous example, if a user whats to understand how the generator function works:

In [0]:
CitrineChallenge.src.tools.generator.generate?

Documentation is available. And for example when they read the documentation, they would see that generator relies on something called the 'switch' function located in src.tools.generator. 

In [0]:
CitrineChallenge.src.tools.generator.switch?

Modularity and documentation assists with the development process such that subsequent work on the library can be initialized quickly. With this, programmers can quicky find how the module works, what it takes as inputs, what it gives as outputs. And with that, they can build updates which fit around the module's paradigm.

---


When it comes to speed, the CitrineChallenge library is pretty fast. Python as an implementation is natively slow because every Python object is really a cleverly disguised C struture, which contains PyObject information on top of the actual value we assigned the object. In short, the benefit of the Python implementation is that it's flexible and dynamic, but all of this extra information which the Python interpereter interacts with ultimately slows down performance.

On previous projects I've worked multiple approaches to mitigate this issue, including integrating C modules directly, building Cython-compiled modules, and using ufuncs in Numpy. Given the time constraint, we used Numpys ufuncs because their performance to development time tradeoff seemed to be the most favorable, specifically in the `switch` function, where linear interpolation is applied between the good and bad points.

To demonstrate, consider the two 1D vectors below, labeled x and y.

In [0]:
npdata = np.random.random_sample((1000,2))

np_x = npdata[:,0]
np_y = npdata[:,1]

As these are the type of vectors which the library is designed to handle, we can pass them directly through the generator function and time the process:

In [0]:
%timeit CitrineChallenge.src.tools.generator.switch(np_x, np_y)

This would compare to the following pure-python implementation, which we could also time:

In [0]:
pydata = [list(x) for x in npdata]

py_x = [x for [x, y] in pydata]
py_y = [y for [x, y] in pydata]

py_pct = np.random.random_sample()

def py_switch(py_x, py_y):
    new_points = [(1-py_pct)*py_x[i] + py_pct*py_y[i] for i in range(len(py_x))]
    return new_points

%timeit py_switch(py_x, py_y)

As we can see, the pure-python implementaiton is about 2 orders of magnitude slower.

We also have the added benefit of numpy's capacity to broadcast the applied mathematics over the given arrays. In python, we would need to apply the `py_switch()` function over each dimension, which would compound the computational time necessary to generate a new vector as its dimensionality is increased.

---

Of all of the objectives listed, vector coverage is the weak spot in the CitrineChallenge implementation. One of the issues here is that the only instance where 'spread' is generated is during the initial random array generation.

In [0]:
url = 'https://raw.githubusercontent.com/1mikegrn/CitrineChallenge/master/tests/mixture.txt'

constraints = requests.get(url).text

class_object = CitrineChallenge.src.constraints.Constraint(
    requests.get(url).text.split('\n')[:-1],
    google=True
)

test_values = np.random.random_sample((250,2))
report_values = np.array([class_object.get_example()])

plt.scatter(test_values[:,0], test_values[:,1])

When we initially generate the vectors, we pass them through the `clean()` function, which separates out the good points from the bad points (and also guarentees legitimate results, satisfying the objective that results must satisfy all constraints). On the below graph, blue points represent vectors which failed the check, and purple points represent vectors that passed.

In [0]:
test_values, report_values = CitrineChallenge.src.tools.cleaning.clean(
    class_object,
    test_values=test_values,
    report_values=report_values
)

plt.scatter(test_values[:,0], test_values[:,1], color='b')
plt.scatter(report_values[:,0], report_values[:,1], color=(1,0,1))

As the program goes through and iteratively replaces the blue points with the purple points (as demonstrated below), we can see the bias that our implementation manifests - as the interpolation process is limited to the space between a good point and a bad point, this means that new data points generated will be, by definition, bound along some axis `x[i]` by the `min(x[i])` and `max(x[i])` vector components generated in the report_values array at initialization. We also have a positive feedback loop here as well, in that, as new results are found  (which are likely biased towards the inner-edge of the feasible region) the probability of the generator picking those inner points for interpolation goes up.

In [0]:
# run the cell to generate the animation. No need to tinker. 

# Note, once run the test_variables and report_variables are overwritten. To 
# build a new animation, first rerun the previous two cells in order, and then
# run this cell.

graph_dict = dict()
iterator = 0

while test_values.shape[0] > 0:

        graph_dict[iterator] = test_values, report_values
        iterator += 1

        # clean test values according to constraints in class_object
        test_values, report_values = CitrineChallenge.src.tools.cleaning.clean(
            class_object=class_object,
            test_values=test_values,
            report_values=report_values
        )

        # calculation is done if test_values is empty
        if test_values.shape[0] == 0:
            break

        # generate new test values by replacing rejected values
        test_values = CitrineChallenge.src.tools.generator.generate(
            test_values=test_values,
            report_values=report_values
        )

graph_dict[iterator] = test_values, report_values

#builds plot
fig, ax = plt.subplots()

# initialize empty
line1, = ax.plot([], [], 'o', color='b')
line2, = ax.plot([], [], 'o', color=(1,0,1))

# initialize empty
def init():
    line1.set_data([], [])
    line2.set_data([], [])

def animate(i):
    line1.set_data(graph_dict[i][0][:,0], graph_dict[i][0][:,1])
    line2.set_data(graph_dict[i][1][:,0], graph_dict[i][1][:,1])

anim = animation.FuncAnimation(
    fig, animate, init_func=init,
    frames=len(graph_dict.keys()), interval=750
)

# don't need to show the figure
plt.close(anim._fig)

# show animation instead
HTML(anim.to_html5_video())

Future versions of the library would probably want to address this. As we can see in this lower-dimensional representation, the implementation isn't *terrible* per se, but this bias compounds as the number of dimensions increases and as the feasible region becomes smaller and more irregular.

For a moment I was considering adding a quick-fix to this, which was to generate something like 10x the number of points requested, and then caluclate the distances between all the points' and then return the top 10% of most-spread out points as the resultant. But during my phone conversation with Sean, he mentioned something summarized to be 'don't be too worried about the bias of an implementation' when comes to the type of programming we're doing here. And, if users wanted to get the fraction of most spread out points in the generated report_values, they could certainly do so pretty easily outside of the library. So, I ultimately decided against hard-coding that idea into the library.

If I were to look into improving this bias in the future though, I would probably look into using the constraints to solve for the edges and corners of the feasible region, and then use those points as mechanisms to drag test_values towards those regions which are unfavored by the current approach. As we can see for the current example, we're constrained by the inequality:

\begin{align*}
1.0 - x[0] - x[1] \geq 0.0
\end{align*}

This constraint builds a feasible region bound by the edges `x[0] + x[1] = 1`, along with `x[0] = 0` and `x[1] = 0` by the definition of the hypercube. A module which built these edge functions and then generated a set of vectors which are on those edges would help normalize the bias of the current implementation.

---


When it's all said and done however, the current library accomplishes its objectives decently well. It's quick, guarentees legitimate results, and is built in a way which would allow for future improvements to the built-in protocols with its modularity and documentation.